In [ ]:
# The process will be this:
# 1. Read all dimensions into pandas dfs
# 2. Per record of the fact load, search in the dimension dataframes for the keys to apply
# 3. write the values of these keys into a new dataframe that will be the final df load
# 4. load adjusted dataframe to MSSQL